In [11]:
import os
from string import Template

In [4]:
Experiment = 'Figure1_Position_Common_Scale'
Datasets = [1, 2, 3]
Classifiers = ['RF_Image', 'RF', 'MLP', 'LeNet', 'VGG19', 'Xception']

In [5]:
NORMALIZATION_FACTOR = '80'

In [8]:
SLURM_DIR = '../SLURMS/' + Experiment
LOG_DIR = SLURM_DIR + '/LOGS/'
if not os.path.exists(SLURM_DIR):
  os.makedirs(SLURM_DIR)
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)

In [18]:
SLURM_TEMPLATE = Template('''#!/bin/bash
#SBATCH -p seas_dgx1
#SBATCH -n 4 # Number of cores
#SBATCH -N 1 # Ensure that all cores are on one machine
#SBATCH --gres=gpu
#SBATCH --mem=32000
#SBATCH -t 3-00:00
#SBATCH --mail-type=ALL
#SBATCH --mail-user=haehn@seas.harvard.edu
#SBATCH -o /n/home05/haehn/Projects/CP/SLURMS/$experiment/LOGS/%N.%j.out
#SBATCH -e /n/home05/haehn/Projects/CP/SLURMS/$experiment/LOGS/%N.%j.err

source new-modules.sh

module load Anaconda/5.0.1-fasrc01

cd /n/home05/haehn/Projects/CP/EXP/

source activate CP

python figure1_experiments.py $experiment $dataset $classifier $normalization_factor

# end
exit 0;
''')

In [19]:
for d in Datasets:
  for c in Classifiers:

      new_slurm = SLURM_TEMPLATE.substitute(experiment=Experiment, dataset=str(d), classifier=c, normalization_factor=NORMALIZATION_FACTOR)
      with open(os.path.join(SLURM_DIR, str(d)+'_'+c+'.sbatch'), 'w') as f:
        f.write(new_slurm)
